In [3]:
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale
from sklearn.preprocessing import normalize

# Feature Extraction

In [4]:
def generate_file_names():
    file_list = []
    for activities in range(19):
        for person in range(8):
            for segments in range(60):
                file_list.append('D:/WSU/Elemnets of netwrok science/project/Graph based semi supervised learning/Data/data/' +
                                 'a' + str(activities + 1).zfill(2) + '/'
                                 'p' + str(person + 1) + '/'
                                 's' + str(segments + 1).zfill(2) + '.txt'
                                )

    return file_list

In [5]:
def extract_features(x):
    numrows = len(x)
    mean = np.mean(x, axis = 0)
    std = np.std(x, axis = 0)
    var = np.var(x, axis = 0)
    median = np.median(x, axis = 0)
    xmax = np.amax(x, axis = 0)
    xmin = np.amin(x, axis = 0)
    p2p = xmax - xmin
    amp = xmax - mean
    s2e = x[numrows-1,:] - x[0,:]
    features = np.concatenate([mean, std, var, median, xmax, xmin, p2p, amp, s2e])
    return features

In [6]:
def data_generate():
    filenames = generate_file_names()
    feature_cols = []
    for unit_label in ['T', 'RA', 'LA', 'RL', 'LL']:
        for feature in ['mean', 'std', 'var', 'median', 'xmax', 'xmin', 'p2p', 'amp', 's2e']:
            for sensor  in ['acc', 'gyro', 'mag']:
                for position in ['X', 'Y', 'Z']:
                    feature_cols.append(feature + '_' + unit_label + '_' + position + sensor)
    activity_data = pd.DataFrame(columns=feature_cols)
    data = []
    activity = []
    person = []
    for i in range(len(filenames)):
        activity.append('a' + str(i // 480 + 1).zfill(2))
        person.append('p' + str((i // 60) % 8 + 1))
        raw_data = np.genfromtxt(filenames[i], delimiter = ',', skip_header = 0)
        row = extract_features(raw_data)
        data.append(row)
    #data = scale(data, axis = 0)
    activity_data = pd.DataFrame(data, columns = feature_cols)
    activity_data['person'] = person
    activity_data['activity'] = activity
    return activity_data

# Convert to Graph

In [7]:
import networkx as nx
from sklearn.preprocessing import normalize

In [8]:
def normalizedData():
    data = pd.read_csv("D:/WSU/Elemnets of netwrok science/project/Graph based semi supervised learning/Data.csv")
    features = data.iloc[:, 1: 46].values
    features_norm = normalize(features)
    return features_norm

In [9]:
features = normalizedData()

In [10]:
from sklearn.metrics.pairwise import *
from sklearn.gaussian_process.kernels import RBF
from sklearn.metrics.pairwise import euclidean_distances

In [14]:
Datacsv = pd.read_csv("D:/WSU/Elemnets of netwrok science/project/Graph based semi supervised learning/Data.csv")
Datacsv

,Unnamed: 0,mean_T_Xacc,mean_T_Yacc,mean_T_Zacc,mean_T_Xgyro,mean_T_Ygyro,mean_T_Zgyro,mean_T_Xmag,mean_T_Ymag,mean_T_Zmag,...,s2e_LL_Yacc,s2e_LL_Zacc,s2e_LL_Xgyro,s2e_LL_Ygyro,s2e_LL_Zgyro,s2e_LL_Xmag,s2e_LL_Ymag,s2e_LL_Zmag,person,activity
0,0,7.975714,1.083150,5.606846,0.004897,0.026123,-0.003726,-0.790726,-0.068490,0.135897,...,0.021200,-0.01950,-0.003672,0.005363,-0.005332,-0.00120,0.002220,0.001468,p1,a01
1,1,7.978250,1.140865,5.598922,0.003308,0.022514,-0.003240,-0.791962,-0.070157,0.137123,...,-0.000600,0.00870,0.002791,-0.014586,-0.015450,-0.00293,0.002020,-0.000020,p1,a01
2,2,7.970894,1.140962,5.607750,0.001846,0.018806,-0.003486,-0.792875,-0.070723,0.136436,...,-0.008800,0.00270,0.003250,-0.003167,0.014497,0.00054,-0.000390,0.001174,p1,a01
3,3,7.938412,1.165260,5.648969,0.003788,0.023768,-0.003512,-0.793550,-0.071969,0.131983,...,0.022000,0.01530,-0.004543,-0.021638,-0.009122,0.00129,-0.000060,0.000816,p1,a01
4,4,7.908930,1.187504,5.680980,0.001086,0.012242,-0.002488,-0.793872,-0.072597,0.128345,...,-0.000800,0.03140,0.014198,0.016549,0.015476,-0.00038,0.000200,-0.000038,p1,a01
5,5,7.965544,1.150181,5.599276,0.002584,0.022276,-0.003579,-0.792878,-0.072232,0.136133,...,-0.015900,-0.01990,0.000014,-0.005488,-0.007275,-0.00075,0.000580,0.000475,p1,a01
6,6,7.923159,1.161473,5.667376,0.003561,0.023468,-0.003175,-0.793819,-0.072314,0.130159,...,-0.001400,0.01790,-0.007411,-0.018780,-0.001869,0.00207,0.002400,0.000776,p1,a01
7,7,7.941025,1.166265,5.642446,0.001864,0.015931,-0.004295,-0.793161,-0.071871,0.133137,...,-0.015600,0.01640,-0.011705,-0.000157,-0.008089,0.00200,-0.001100,0.001259,p1,a01
8,8,7.882650,1.179440,5.720484,0.004575,0.023619,-0.003718,-0.793967,-0.073239,0.124560,...,-0.015700,0.01670,0.010712,0.007350,0.004563,0.00120,-0.000310,-0.000376,p1,a01
9,9,7.902089,1.212818,5.689686,0.002180,0.017235,-0.002418,-0.793208,-0.073189,0.128213,...,-0.000600,0.00880,0.003452,-0.009715,0.003576,0.00002,-0.000190,-0.000151,p1,a01


In [11]:
euclidean_distances(features)[0][1]

0.00673729591434673

In [86]:
features.shape

(9120, 45)

In [13]:
import networkx as nx
import csv

Data  = open('D:/WSU/Elemnets of netwrok science/project/Graph based semi supervised learning/Data.csv', "r", encoding='utf8')
next(Data, None)
read = csv.reader(Data)
  # skip the first line in the input file
Graphtype = nx.Graph()

G = nx.parse_edgelist(Data, delimiter=',', create_using=Graphtype,
                      nodetype=int, data=(('weight', float),))
   # use net.Graph() for undirected graph


for x in G.nodes():
      print ("Node:", x, "has total #degree:",G.degree(x), " , In_degree: ", G.out_degree(x)," and out_degree: ", G.in_degree(x))   
for u,v in G.edges():
      print ("Weight of Edge ("+str(u)+","+str(v)+")", G.get_edge_data(u,v))

nx.draw(G)
plt.show()

TypeError: Failed to convert nodes 0,7.975714400000001 to type <class 'int'>.

In [7]:
import csv
import random
import math
import operator
 
def loadDataset(filename, split, trainingSet=[] , testSet=[]):
    with open(filename, 'r') as csvfile:
        lines = csv.reader(csvfile)
        dataset = list(lines)
        for x in range(2, 120):
            for y in range(406):
                dataset[x][y] = float(dataset[x][y])
                if random.random() < split:
                    trainingSet.append(dataset[x])
                else:
                    testSet.append(dataset[x])
 
 


In [8]:
def euclideanDistance(instance1, instance2, length):
    distance = 0
    for x in range(length):
        distance += pow((instance1[x] - instance2[x]), 2)
        return math.sqrt(distance)
 


In [9]:
def getNeighbors(trainingSet, testInstance, k):
    distances = []
    length = len(testInstance)-1
    for x in range(len(trainingSet)):
        dist = euclideanDistance(testInstance, trainingSet[x], length)
        distances.append((trainingSet[x], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors
 


In [10]:
def getResponse(neighbors):
    classVotes = {}
    for x in range(len(neighbors)):
        response = neighbors[x][-1]
        if response in classVotes:
            classVotes[response] += 1
        else:
            classVotes[response] = 1
    sortedVotes = sorted(classVotes.items(), key=operator.itemgetter(1), reverse=True)
    return sortedVotes[0][0]
 


In [11]:
def getAccuracy(testSet, predictions):
    correct = 0
    for x in range(len(testSet)):
        if testSet[x][-1] == predictions[x]:
            correct += 1
    return (correct/float(len(testSet))) * 100.0
    


In [12]:
def main():
    # prepare data
    trainingSet=[]
    testSet=[]
    split = 0.67
    loadDataset("D:/WSU/Elemnets of netwrok science/project/Graph based semi supervised learning/Data.csv", split, trainingSet, testSet)
    print('Train set: ' + repr(len(trainingSet)))
    print('Test set: ' + repr(len(testSet)))
    # generate predictions
    predictions=[]
    k = 3
    for x in range(len(testSet)):
        neighbors = getNeighbors(trainingSet, testSet[x], k)
        result = getResponse(neighbors)
        predictions.append(result)
        #print('> predicted=' + repr(result) + ', actual=' + repr(testSet[x][-1]))
    accuracy = getAccuracy(testSet, predictions)
    print('Accuracy: ' + repr(accuracy) + '%')
    
main()

Train set: 31949
Test set: 15959
Accuracy: 100.0%
